Testing a Semi-Definite Program.

A semidefinite program (SDP) is an optimization problem of the form
$$
\text{minimize   }  tr(CX) \\
\text{subject to  } tr(A_iX) = b_i, i = 1,...,p \\
             X >= 0, \\
$$

In [95]:
# Import packages.
import cvxpy as cp 
import numpy as np
import math
import matlab
import random

In [89]:
c = 0
I = np.identity(2)
# e4 = np.matrix([[-1/3], [-math.sqrt(2)/3], [-math.sqrt(2/3)]])

basis = []


def generate_projections(basis):
    
    projections = []
    for e in basis:
        projections.append(np.matmul(e, e.H))
    
    return projections

def probability(state, E1, E2):
    
    rho = np.kron(E1, E2)
    return np.matmul(np.matmul(state.H, rho), state)


def generate_box(state, P1, P2):
    Q1 = I - P1
    Q2 = I - P2
    
    results = []
    for Alice in [P1, Q1]:
        for Bob in [P2, Q2]:
            results.append(probability(state, Alice, Bob))
    
    return results

def normalize_vector(vector):
    # Calculate the magnitude of the vector
    magnitude = np.linalg.norm(vector)
    # Divide each component by the magnitude
    return vector / magnitude


def add_perturbation(basis, epsilon):
    
    theta = epsilon * (np.random.rand() - 0.5)
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])
    
    return np.dot(rotation_matrix, basis)

def get_box(perturbation):
    
    # theta = perturbation
    # rotation_matrix = np.array([
    #     [np.cos(theta), -np.sin(theta)],
    #     [np.sin(theta), np.cos(theta)]
    # ])
    
    e1 = normalize_vector(add_perturbation(np.matrix([[1],[0]]), perturbation))
    e2 = normalize_vector(add_perturbation(np.matrix([[1/2], [-(math.sqrt(3))/2]]), perturbation))
    e3 = normalize_vector(add_perturbation(np.matrix([[-1/2], [-math.sqrt(3)/2]]), perturbation))
    
    
    projections = generate_projections([e1, e2, e3])

    ket0 = np.matrix([[1], [0]])
    ket1 = np.matrix([[0], [1]])
    
    ket00 = np.kron(ket0, ket0)
    ket11 = np.kron(ket1, ket1)
    
    state = np.dot(ket00 + ket11, 1/math.sqrt(2))

    box = [[0 for i in range(6)] for j in range(6)]
    
    for i in range(1, len(projections) + 1):
        for j in range(1, len(projections) + 1):
            A = projections[i - 1]
            B = projections[j - 1]
            results = generate_box(state, A, B)
            
            x = i - 1
            y = j - 1
            
            box[2 * x][2 * y]           = results[0]
            box[2 * x][2 * y + 1]       = results[1]
            box[2 * x + 1][2 * y]       = results[2]
            box[2 * x + 1][2 * y + 1]   = results[3]
    
    return box

box = get_box(0)
for row in box:
    for col in row:
        print(f"{col[(0, 0)]:>9.3f}", end=" ")  # Use a single space
    print()


    0.500     0.000     0.125     0.375     0.125     0.375 
    0.000     0.500     0.375     0.125     0.375     0.125 
    0.125     0.375     0.500    -0.000     0.125     0.375 
    0.375     0.125    -0.000     0.500     0.375     0.125 
    0.125     0.375     0.125     0.375     0.500    -0.000 
    0.375     0.125     0.375     0.125    -0.000     0.500 


We can generate the quantum probability distribution using the simplex vector with the above code.
Below code is used to perturbate the Probability Distribution within the following constraints:
1. No-Signaling Constraints
2. All probabilities are non-negative.
3. Probabilities are normalized.

In [100]:
d = 2

def perturb_box(epsilon, box=[]):
    
    prob_dist = np.array([
    [0.500, 0.000, 0.125, 0.375, 0.125, 0.375],
    [0.000, 0.500, 0.375, 0.125, 0.375, 0.125],
    [0.125, 0.375, 0.500, 0.000, 0.125, 0.375],
    [0.375, 0.125, 0.000, 0.500, 0.375, 0.125],
    [0.125, 0.375, 0.125, 0.375, 0.500, 0.000],
    [0.375, 0.125, 0.375, 0.125, 0.000, 0.500]
    ])
    
    # e = -epsilon
    perturbation = np.zeros_like(prob_dist)
    
    for x in range(1):
        for y in range(3):
            i = 2 * x
            j = 2 * y
            
            e11ij = random.uniform(0, epsilon)
            e12ij = epsilon - e11ij
            e22ij = -epsilon
            e21ij = 0
            
            perturbation[i + 0][j + 0] = e11ij
            perturbation[i + 0][j + 1] = e12ij
            perturbation[i + 1][j + 0] = e21ij
            perturbation[i + 1][j + 1] = e22ij

            print(perturbation[i][j] + perturbation[i + 1][j] + perturbation[i][j + 1] + perturbation[i + 1][j + 1])
    
    for row in perturbation:
        print(row)
    
            
            
perturb_box(0.1)


    
    

0.0
0.0
0.0
[0.0780302  0.0219698  0.02898658 0.07101342 0.06176367 0.03823633]
[ 0.  -0.1  0.  -0.1  0.  -0.1]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]


In [54]:
# Find all classical strategies for d

d = 2
# There will be d + 1 projections.

# There are 3 possible inputs per party and 2 possible outputs.
# inputs = {0, 1, 2} and outputs = {0, 1}
# 2^3 possible strategy per party, so 2^3 * 2^3 = 64 possible strategies
# Due to symmetricity of simplex vectors, e.g. (1, 0, 0) = (0, 1, 0) = (0, 0, 1)
# there will be 16 unique strategies


def find_strats(x, n):
    if n == 0:
        return x
    
    new = []
    for partial_strat in x:
        new.append(partial_strat + [0])
        new.append(partial_strat + [1])
    
    if not new:
        new = [[0], [1]]
    
    return find_strats(new, n - 1)

def generate_Pli(A, B):
    
    local_strat = np.zeros((2 * (d + 1), 2 * (d + 1)))
    
    for X in range(d + 1):
        for Y in range(d + 1):
            a = A[X]
            b = B[Y]
            
            local_strat[2 * X + a][2 * Y + b] = 1
    for l in local_strat:
        print(l)
    print('==============')
    return local_strat

def generate_classical_strats():
    result = []
    
    for A in find_strats([], d + 1):
        for B in find_strats([], d + 1):
            result.append(generate_Pli(A, B))
    
    return result

P_L = generate_classical_strats()



[1. 0. 1. 0. 1. 0.]
[0. 0. 0. 0. 0. 0.]
[1. 0. 1. 0. 1. 0.]
[0. 0. 0. 0. 0. 0.]
[1. 0. 1. 0. 1. 0.]
[0. 0. 0. 0. 0. 0.]
[1. 0. 1. 0. 0. 1.]
[0. 0. 0. 0. 0. 0.]
[1. 0. 1. 0. 0. 1.]
[0. 0. 0. 0. 0. 0.]
[1. 0. 1. 0. 0. 1.]
[0. 0. 0. 0. 0. 0.]
[1. 0. 0. 1. 1. 0.]
[0. 0. 0. 0. 0. 0.]
[1. 0. 0. 1. 1. 0.]
[0. 0. 0. 0. 0. 0.]
[1. 0. 0. 1. 1. 0.]
[0. 0. 0. 0. 0. 0.]
[1. 0. 0. 1. 0. 1.]
[0. 0. 0. 0. 0. 0.]
[1. 0. 0. 1. 0. 1.]
[0. 0. 0. 0. 0. 0.]
[1. 0. 0. 1. 0. 1.]
[0. 0. 0. 0. 0. 0.]
[0. 1. 1. 0. 1. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 1. 1. 0. 1. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 1. 1. 0. 1. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 1. 1. 0. 0. 1.]
[0. 0. 0. 0. 0. 0.]
[0. 1. 1. 0. 0. 1.]
[0. 0. 0. 0. 0. 0.]
[0. 1. 1. 0. 0. 1.]
[0. 0. 0. 0. 0. 0.]
[0. 1. 0. 1. 1. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 1. 0. 1. 1. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 1. 0. 1. 1. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 1. 0. 1. 0. 1.]
[0. 0. 0. 0. 0. 0.]
[0. 1. 0. 1. 0. 1.]
[0. 0. 0. 0. 0. 0.]
[0. 1. 0. 1. 0. 1.]
[0. 0. 0. 0. 0. 0.]
[1. 0. 1. 0. 1. 0.]
[0. 0. 0. 0. 0. 0.]


min tr((M^T)p_q) 

subject to tr((M^T)P^{(L,i)}) >= 1 for all i

            M >= 0

In [85]:

def generate_simplex_q(epsilon):
    Q = np.zeros((2 * (d + 1), 2 * (d + 1)))

    Pq = get_box(epsilon)
    
    for i in range(2 * (d + 1)):
        for j in range(2 * (d + 1)):
            a = i % 2
            b = j % 2
            x = i // 2
            y = j // 2
            
            if a == b == 0 and x == y:
                # Q[i][j] = 1 / d
                Q[i][j] = Pq[i][j]
            elif a == b == 1 and x == y:
                # Q[i][j] = 1 - (1/d)
                Q[i][j] = Pq[i][j]
            elif a == b == 0 and x != y:
                # Q[i][j] = 1 / (d**3)
                Q[i][j] = Pq[i][j]
            elif (a + 1) % 2 == b and x != y:
                # Q[i][j] = (d**2 - 1)/(d ** 3)
                Q[i][j] = Pq[i][j]
            elif a == b == 1 and x != y:
                # Q[i][j] = (d**3 - 2*(d**2) + 1)/(d ** 3)
                Q[i][j] = Pq[i][j]
            else:
                # Q[i][j] = 0.0
                Q[i][j] = Pq[i][j]
    return Q

def optimize_coefficients(limit=-1):
        
    Q = generate_simplex_q(0)
    # Q = get_box(0)


    M = cp.Variable((2 * (d + 1), 2 * (d + 1)), symmetric=True)
    
    constraints = [M >= 0]
    constraints += [
        cp.trace(M.T @ P_L[i]) <= 1 for i in range(len(P_L))
    ]
    
    if limit != -1:
        constraints += [
            cp.trace(M.T @ Q) >= limit
        ]
    
    prob = cp.Problem(cp.Maximize(cp.trace(M.T @ Q)), constraints)
    prob.solve()

    print("The optimal value is", prob.value)

    return M


M = optimize_coefficients()
matlab_array = matlab.double(M.value)
# We check for perturbations
print(M.value)
e = -12
mini = float('-inf')
e_mini = None
while e <= 12:
    Q = generate_simplex_q(e)
    x = cp.trace(M.T @ Q).value
    print(x, e)
    
    if x > mini:
        mini = max(x, mini)
        e_mini = e
    
    e += 0.01
    
        
print("-----------------------------")
print(mini, e_mini)

    

The optimal value is 1.0833333333177655
[[1.11111104e-01 3.58022225e-12 3.08388859e-11 1.66666667e-01
  3.08388859e-11 1.66666670e-01]
 [3.58022225e-12 1.11111110e-01 1.66666667e-01 3.08388859e-11
  1.66666664e-01 3.08388859e-11]
 [3.08388859e-11 1.66666667e-01 1.11111112e-01 3.58022225e-12
  3.08388859e-11 1.66666666e-01]
 [1.66666667e-01 3.08388859e-11 3.58022225e-12 1.11111111e-01
  1.66666667e-01 3.08388859e-11]
 [3.08388859e-11 1.66666664e-01 3.08388859e-11 1.66666667e-01
  1.11111117e-01 3.58022225e-12]
 [1.66666670e-01 3.08388859e-11 1.66666666e-01 3.08388859e-11
  3.58022225e-12 1.11111112e-01]]
0.958356893903201 -12
0.9862833125107516 -11.99
0.9805916649493576 -11.98
0.7619490492600121 -11.97
1.001886758939886 -11.96
0.6242662533448224 -11.950000000000001
0.9392833642708935 -11.940000000000001
0.9401662041095841 -11.930000000000001
0.8216071895061443 -11.920000000000002
0.9981756834794816 -11.910000000000002
0.40208832864025323 -11.900000000000002
0.9900043669925899 -11.890000

In [81]:
import matlab.engine
eng = matlab.engine.start_matlab()

eng.addpath(r'/Users/jamkabeeralikhan/Documents/MATLAB')

def bellMax(d, arr):

    result = eng.Optimizer(d, matlab_array)

    # Print the result
    print(f"The result is: {result}")

print(bellMax(d, matlab_array))

# Close the MATLAB engine
eng.quit()


(:,:,1,1) =

    0.1111    0.0000
    0.0000    0.1111


(:,:,2,1) =

    0.0000    0.1667
    0.1667    0.0000


(:,:,3,1) =

    0.0000    0.1667
    0.1667    0.0000


(:,:,1,2) =

    0.0000    0.1667
    0.1667    0.0000


(:,:,2,2) =

    0.1111    0.0000
    0.0000    0.1111


(:,:,3,2) =

    0.0000    0.1667
    0.1667    0.0000


(:,:,1,3) =

    0.0000    0.1667
    0.1667    0.0000


(:,:,2,3) =

    0.0000    0.1667
    0.1667    0.0000


(:,:,3,3) =

    0.1111    0.0000
    0.0000    0.1111

    1.0000

    1.3333

The result is: 1.0833333329758332
None
